In [1]:
# !pip install geopandas
# !pip install mapclassify
# !pip install rasterio

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import geopandas as gp
import folium
import geopandas
import mapclassify
import zipfile
import requests
import xml.etree.ElementTree as ET
import rasterio
from rasterio.plot import show
from tqdm import tqdm

In [3]:
!mkdir 'fip_tif'

mkdir: cannot create directory ‘fip_tif’: File exists


In [4]:
# skip is the last county already downloaded
num_weeks = 376
skip = 67
base = datetime.date.fromisoformat('2022-02-22')
date_list = [base - datetime.timedelta(weeks=(x)) for x in range(num_weeks)]

# add 0-padding to county FIPS
county_list = pd.read_csv('https://raw.githubusercontent.com/hohohohia/misc/main/County_450.csv')['county'].astype(str)[skip:]
county_list = [x.zfill(5) for x in county_list]

# Number of counties left in list
print(len(county_list),'counties left')

383 counties left


In [14]:
county_list1 = county_list[:100]
county_list2 = county_list[100:200]
county_list3 = county_list[200:300]
county_list4 = county_list[300:]

In [15]:
def get_SMAP_image(date, fip):
    fip = fip
    date = date
    date_str = datetime.datetime.strptime(date, '%Y.%m.%d').date().strftime('%Y-%m-%d')

    url = f'https://cloud.csiss.gmu.edu/smap_service?service=WPS&version=1.0.0&request=Execute&identifier=GetFileByFips&DataInputs=layer=SMAP-9KM-DAILY-SUB_{date}_AVERAGE;fips={fip}'

    r = requests.get(url)

    response_xml_as_string = r.text
    responseXml = ET.fromstring(response_xml_as_string)

    tif_url = responseXml[2][0][3][0].text
    
    !wget $tif_url -O $'fip_tif/{fip}_{date_str}.tif' -q --no-check-certificate
    # !cp 'fip_tif/06019_2019-01-01.tif' '/content/gdrive/My Drive/fip_tif/06019_2019-01-01.tif'


    return rasterio.open(f'fip_tif/{fip}_{date_str}.tif').read()

In [ ]:
data_by_FIP = pd.DataFrame()
data_by_FIP['Dates'] = date_list
temp_data = []

for j, county in enumerate(county_list1):
    for i in tqdm(range(num_weeks)):
        try:
            im = get_SMAP_image(date_list[i].strftime('%Y.%m.%d'), county)
            temp_data.append(im[im > -9999].mean())
        except:
            temp_data.append(-9999)
            continue
    data_by_FIP[county] = temp_data
    data_by_FIP.to_csv(f'smap_test_avgs_{j+skip}_counties.csv')
    temp_data = []


 24%|██▍       | 91/376 [00:43<02:19,  2.04it/s]